### Predict Response Time with Michael's Accessibility Score

---

* Objective: 
  * Does Accessibility matter when predicting response time?

* Notes/Scope
 * Make sure I aggregate features
 * Use Michaels v4 data
---
* __author__: Eric 
* __credits__: Michael & Jude
* __status__: Development

In [2]:
#packages

import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
import matplotlib as plt

import warnings
import sqlite3

from sklearn import preprocessing
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import accuracy_score

# hide warnings
warnings.filterwarnings('ignore')

In [3]:
# improvement from earlier process and use code Jude wrote to connect to v4 data
def create_connection(path):
    connection = None 
    connection = sqlite3.connect(path)
    connection.text_factory = str

    return connection

# make sure that the fire data has been downloaded into the "/data" directory in the project root
con = create_connection('../data/fire_data_v4.db')

## Load and transform the data for the model

In [4]:

cur = con.cursor()

truck_and_engine_calls = pd.read_sql("""
        select 
          calls_for_service.call_number
        , calls_for_service.incident_number
        , floating_catchment_output.[index]
        , floating_catchment_output.zone_idx
        , floating_catchment_output.accessibility_score
        , floating_catchment_output.scenario_name        
        , calls_for_service.on_scene_dttm
        , calls_for_service.response_dttm  
        , calls_for_service.zipcode_of_incident
        , fi.neighborhood_district
        , category_mappings.[index] as primary_situation_index
        from floating_catchment_output
         inner join zone_idx_to_incident 
          on zone_idx_to_incident.zone_idx = floating_catchment_output.zone_idx
         inner join calls_for_service 
          on calls_for_service.incident_number = zone_idx_to_incident.incident_number
         inner join fire_incidents AS fi 
          on calls_for_service.incident_number = fi.incident_number
         left join category_mappings 
          on fi.primary_situation = category_mappings.primary_situation
        where 
         calls_for_service.unit_type in ('TRUCK', 'ENGINE')
         and scenario_name = 'baseline'
        """,con = con)

truck_and_engine_calls.shape
truck_and_engine_calls.head()

,call_number,incident_number,index,zone_idx,accessibility_score,scenario_name,on_scene_dttm,response_dttm,zipcode_of_incident,neighborhood_district,primary_situation_index
0,190060098,19002211,0,8,0.000803,baseline,2019-01-06 00:54:34,01/06/2019 12:52:49 AM,94121.0,Outer Richmond,148.0
1,190060098,19002211,0,8,0.000803,baseline,2019-01-06 00:57:30,01/06/2019 12:53:23 AM,94121.0,Outer Richmond,148.0
2,190141726,19005740,0,8,0.000803,baseline,2019-01-14 12:32:43,01/14/2019 12:25:54 PM,94121.0,Outer Richmond,81.0
3,190141726,19005740,0,8,0.000803,baseline,None,01/14/2019 12:31:39 PM,94121.0,Outer Richmond,81.0
4,190141726,19005740,0,8,0.000803,baseline,None,01/14/2019 12:26:17 PM,94121.0,Outer Richmond,81.0


In [5]:

# create some additional features
truck_and_engine_calls['on_scene_dttm'] = pd.to_datetime(truck_and_engine_calls['on_scene_dttm'])
truck_and_engine_calls['response_dttm'] = pd.to_datetime(truck_and_engine_calls['response_dttm'])

truck_and_engine_calls['hour_of_day'] = truck_and_engine_calls.response_dttm.dt.hour

truck_and_engine_calls['arrival_time'] = (truck_and_engine_calls['on_scene_dttm'] - truck_and_engine_calls['response_dttm'])
truck_and_engine_calls['minutes'] = (truck_and_engine_calls.arrival_time.dt.seconds) / 60
truck_and_engine_calls['seconds'] = (truck_and_engine_calls.arrival_time.dt.seconds)

truck_and_engine_calls['date'] = truck_and_engine_calls['on_scene_dttm'].dt.date
truck_and_engine_calls['date'] = pd.to_datetime(truck_and_engine_calls['date'])
truck_and_engine_calls['day_of_week'] = truck_and_engine_calls['on_scene_dttm'].dt.day_name().astype(str)
truck_and_engine_calls['dayflag'] = (truck_and_engine_calls.on_scene_dttm.dt.hour > 5) & (truck_and_engine_calls.on_scene_dttm.dt.hour <18)
truck_and_engine_calls['week_number'] = truck_and_engine_calls['on_scene_dttm'].dt.week
truck_and_engine_calls['day_of_month'] = truck_and_engine_calls['on_scene_dttm'].dt.day
truck_and_engine_calls['year'] = truck_and_engine_calls['on_scene_dttm'].dt.year

# holiday check
cal = calendar()
dt = pd.date_range(start=truck_and_engine_calls['date'].min(), end=truck_and_engine_calls['date'].max())
holidays = cal.holidays(start=dt.min(), end=dt.max())
truck_and_engine_calls['holiday_flag'] = truck_and_engine_calls['date'].isin(holidays)

pd.options.display.max_columns = 35
truck_and_engine_calls.head()

,call_number,incident_number,index,zone_idx,accessibility_score,scenario_name,on_scene_dttm,response_dttm,zipcode_of_incident,neighborhood_district,primary_situation_index,hour_of_day,arrival_time,minutes,seconds,date,day_of_week,dayflag,week_number,day_of_month,year,holiday_flag
0,190060098,19002211,0,8,0.000803,baseline,2019-01-06 00:54:34,2019-01-06 00:52:49,94121.0,Outer Richmond,148.0,0.0,0 days 00:01:45,1.750000,105.0,2019-01-06,Sunday,False,1.0,6.0,2019.0,False
1,190060098,19002211,0,8,0.000803,baseline,2019-01-06 00:57:30,2019-01-06 00:53:23,94121.0,Outer Richmond,148.0,0.0,0 days 00:04:07,4.116667,247.0,2019-01-06,Sunday,False,1.0,6.0,2019.0,False
2,190141726,19005740,0,8,0.000803,baseline,2019-01-14 12:32:43,2019-01-14 12:25:54,94121.0,Outer Richmond,81.0,12.0,0 days 00:06:49,6.816667,409.0,2019-01-14,Monday,True,3.0,14.0,2019.0,False
3,190141726,19005740,0,8,0.000803,baseline,NaT,2019-01-14 12:31:39,94121.0,Outer Richmond,81.0,12.0,NaT,NaN,NaN,NaT,nan,False,NaN,NaN,NaN,False
4,190141726,19005740,0,8,0.000803,baseline,NaT,2019-01-14 12:26:17,94121.0,Outer Richmond,81.0,12.0,NaT,NaN,NaN,NaT,nan,False,NaN,NaN,NaN,False


In [6]:
# drop incidents where the truck or engine responded but never arrived
incident_first_response = truck_and_engine_calls.loc[~pd.isna(truck_and_engine_calls.seconds)]

# find the minimum, average, and maximum travel/response times for each incident
incident_first_response = incident_first_response.groupby("incident_number").agg({'seconds': ['min', 'mean', 'max']})
incident_first_response = incident_first_response.reset_index(col_level=1)
incident_first_response.columns = list(map(lambda x: x[1] if x[0] == "" else x[0]+"_"+x[1], incident_first_response.columns))
incident_first_response = incident_first_response.set_index("incident_number")
incident_first_response.head()

,seconds_min,seconds_mean,seconds_max
incident_number,,,
19000005,238.0,637.0,1458.0
19000013,113.0,113.0,113.0
19000018,134.0,199.0,264.0
19000023,199.0,246.5,294.0
19000025,145.0,145.0,145.0


In [7]:
# build the modeling dataframe
model_data = truck_and_engine_calls.loc[:,['incident_number', 'neighborhood_district', 'accessibility_score', 'day_of_week', 'hour_of_day']]
# ensure that we only have once incident in the frame
model_data = model_data.groupby(['incident_number', 'neighborhood_district', 'accessibility_score']).agg({
    'day_of_week' : 'min',
    'hour_of_day' : 'min'
})
model_data = model_data.reset_index()
model_data = model_data.set_index("incident_number")
model_data = model_data.join(incident_first_response, how="inner")
model_data.head()

,neighborhood_district,accessibility_score,day_of_week,hour_of_day,seconds_min,seconds_mean,seconds_max
incident_number,,,,,,,
19000005,West of Twin Peaks,0.000401,Tuesday,0.0,238.0,637.0,1458.0
19000013,Inner Sunset,0.000549,Tuesday,0.0,113.0,113.0,113.0
19000018,Inner Richmond,0.000515,Tuesday,0.0,134.0,199.0,264.0
19000023,Bayview Hunters Point,0.000527,Tuesday,0.0,199.0,246.5,294.0
19000025,Haight Ashbury,0.000268,Tuesday,0.0,145.0,145.0,145.0


In [40]:
model_data_agg = model_data.groupby(['neighborhood_district', "accessibility_score"]).agg({"seconds_min":"mean"})
model_data_agg = model_data_agg.reset_index()
model_data_agg.head()

,neighborhood_district,accessibility_score,seconds_min
0,Bayview Hunters Point,0.000097,175.377143
1,Bayview Hunters Point,0.000196,230.316667
2,Bayview Hunters Point,0.000204,157.000000
3,Bayview Hunters Point,0.000254,252.370370
4,Bayview Hunters Point,0.000266,475.750000


## Overall Correlation (all times & days)

In [41]:
pd.set_option('display.max_rows', 82)
model_data_agg.loc[:, ['accessibility_score', 'seconds_min', 'neighborhood_district']].groupby('neighborhood_district').corr(method="spearman")

accessibility_score  \
neighborhood_district                                                     
Bayview Hunters Point          accessibility_score             1.000000   
                               seconds_min                     0.217391   
Bernal Heights                 accessibility_score             1.000000   
                               seconds_min                     0.006993   
Castro/Upper Market            accessibility_score             1.000000   
                               seconds_min                     0.159341   
Chinatown                      accessibility_score             1.000000   
                               seconds_min                    -0.900000   
Excelsior                      accessibility_score             1.000000   
                               seconds_min                     0.116667   
Financial District/South Beach accessibility_score             1.000000   
                               seconds_min                     0.174825   
Glen Park                      accessibility_score             1.000000   
                               seconds_min                     0.485714   
Golden Gate Park               accessibility_score             1.000000   
                               seconds_min                     0.450549   
Haight Ashbury                 accessibility_score             1.000000   
                               seconds_min                    -0.018182   
Hayes Valley                   accessibility_score             1.000000   
                               seconds_min                    -0.660606   
Inner Richmond                 accessibility_score             1.000000   
                               seconds_min                    -0.771429   
Inner Sunset                   accessibility_score             1.000000   
                               seconds_min                     0.700000   
Japantown                      accessibility_score             1.000000   
                               seconds_min                    -0.600000   
Lakeshore                      accessibility_score             1.000000   
                               seconds_min                    -0.100000   
Lincoln Park                   accessibility_score             1.000000   
                               seconds_min                    -1.000000   
Lone Mountain/USF              accessibility_score             1.000000   
                               seconds_min                     0.214286   
Marina                         accessibility_score             1.000000   
                               seconds_min                    -0.142857   
McLaren Park                   accessibility_score             1.000000   
                               seconds_min                     0.200000   
Mission                        accessibility_score             1.000000   
                               seconds_min                     0.159398   
Mission Bay                    accessibility_score             1.000000   
                               seconds_min                    -0.054545   
Nob Hill                       accessibility_score             1.000000   
                               seconds_min                     0.483333   
Noe Valley                     accessibility_score             1.000000   
                               seconds_min                     0.900000   
North Beach                    accessibility_score             1.000000   
                               seconds_min                    -0.400000   
Oceanview/Merced/Ingleside     accessibility_score             1.000000   
                               seconds_min                    -0.314286   
Outer Mission                  accessibility_score             1.000000   
                               seconds_min                    -0.104895   
Outer Richmond                 accessibility_score             1.000000   
                               seconds_min                     0.350000   
Pa

## Non-Rushhour correlations

In [43]:
model_data_agg = model_data.loc[model_data.hour_of_day.ge(10) & model_data.hour_of_day.le(16), :].groupby(['neighborhood_district', "accessibility_score"]).agg({"seconds_min":"mean"})
model_data_agg = model_data_agg.reset_index()
pd.set_option('display.max_rows', 82)
model_data_agg.loc[:, ['accessibility_score', 'seconds_min', 'neighborhood_district']].groupby('neighborhood_district').corr(method="spearman")

accessibility_score  \
neighborhood_district                                                     
Bayview Hunters Point          accessibility_score             1.000000   
                               seconds_min                     0.198193   
Bernal Heights                 accessibility_score             1.000000   
                               seconds_min                     0.272727   
Castro/Upper Market            accessibility_score             1.000000   
                               seconds_min                     0.263736   
Chinatown                      accessibility_score             1.000000   
                               seconds_min                     0.300000   
Excelsior                      accessibility_score             1.000000   
                               seconds_min                     0.142857   
Financial District/South Beach accessibility_score             1.000000   
                               seconds_min                    -0.181818   
Glen Park                      accessibility_score             1.000000   
                               seconds_min                    -0.142857   
Golden Gate Park               accessibility_score             1.000000   
                               seconds_min                     0.483516   
Haight Ashbury                 accessibility_score             1.000000   
                               seconds_min                    -0.539394   
Hayes Valley                   accessibility_score             1.000000   
                               seconds_min                     0.116667   
Inner Richmond                 accessibility_score             1.000000   
                               seconds_min                    -0.500000   
Inner Sunset                   accessibility_score             1.000000   
                               seconds_min                     0.116667   
Japantown                      accessibility_score             1.000000   
                               seconds_min                    -1.000000   
Lakeshore                      accessibility_score             1.000000   
                               seconds_min                    -0.100000   
Lincoln Park                   accessibility_score                  NaN   
                               seconds_min                          NaN   
Lone Mountain/USF              accessibility_score             1.000000   
                               seconds_min                     0.535714   
Marina                         accessibility_score             1.000000   
                               seconds_min                    -0.321429   
McLaren Park                   accessibility_score             1.000000   
                               seconds_min                     0.400000   
Mission                        accessibility_score             1.000000   
                               seconds_min                     0.292857   
Mission Bay                    accessibility_score             1.000000   
                               seconds_min                    -0.142857   
Nob Hill                       accessibility_score             1.000000   
                               seconds_min                     0.150000   
Noe Valley                     accessibility_score             1.000000   
                               seconds_min                     0.233333   
North Beach                    accessibility_score             1.000000   
                               seconds_min                    -0.800000   
Oceanview/Merced/Ingleside     accessibility_score             1.000000   
                               seconds_min                    -0.300000   
Outer Mission                  accessibility_score             1.000000   
                               seconds_min                    -0.018182   
Outer Richmond                 accessibility_score             1.000000   
                               seconds_min                    -0.166667   
Pa

,accessibility_score,seconds_min
accessibility_score,1.000000,0.176705
seconds_min,0.176705,1.000000


accessibility_score  \
neighborhood_district                                                     
Bayview Hunters Point          accessibility_score             1.000000   
                               seconds_min                     0.198193   
Bernal Heights                 accessibility_score             1.000000   
                               seconds_min                     0.272727   
Castro/Upper Market            accessibility_score             1.000000   
                               seconds_min                     0.263736   
Chinatown                      accessibility_score             1.000000   
                               seconds_min                     0.300000   
Excelsior                      accessibility_score             1.000000   
                               seconds_min                     0.142857   
Financial District/South Beach accessibility_score             1.000000   
                               seconds_min                    -0.181818   
Glen Park                      accessibility_score             1.000000   
                               seconds_min                    -0.142857   
Golden Gate Park               accessibility_score             1.000000   
                               seconds_min                     0.483516   
Haight Ashbury                 accessibility_score             1.000000   
                               seconds_min                    -0.539394   
Hayes Valley                   accessibility_score             1.000000   
                               seconds_min                     0.116667   
Inner Richmond                 accessibility_score             1.000000   
                               seconds_min                    -0.500000   
Inner Sunset                   accessibility_score             1.000000   
                               seconds_min                     0.116667   
Japantown                      accessibility_score             1.000000   
                               seconds_min                    -1.000000   
Lakeshore                      accessibility_score             1.000000   
                               seconds_min                    -0.100000   
Lincoln Park                   accessibility_score                  NaN   
                               seconds_min                          NaN   
Lone Mountain/USF              accessibility_score             1.000000   
                               seconds_min                     0.535714   
Marina                         accessibility_score             1.000000   
                               seconds_min                    -0.321429   
McLaren Park                   accessibility_score             1.000000   
                               seconds_min                     0.400000   
Mission                        accessibility_score             1.000000   
                               seconds_min                     0.292857   
Mission Bay                    accessibility_score             1.000000   
                               seconds_min                    -0.142857   
Nob Hill                       accessibility_score             1.000000   
                               seconds_min                     0.150000   
Noe Valley                     accessibility_score             1.000000   
                               seconds_min                     0.233333   
North Beach                    accessibility_score             1.000000   
                               seconds_min                    -0.800000   
Oceanview/Merced/Ingleside     accessibility_score             1.000000   
                               seconds_min                    -0.300000   
Outer Mission                  accessibility_score             1.000000   
                               seconds_min                    -0.018182   
Outer Richmond                 accessibility_score             1.000000   
                               seconds_min                    -0.166667   
Pa